In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import requests
import pandas as pd
import re
from selenium.common.exceptions import UnexpectedAlertPresentException
import urllib.request
import random
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import logging


### 키워드 설정

In [ ]:
keyword = ['샘플입니당']

### 로그인 설정

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 크롬 드라이버 제어
#driver = webdriver.Chrome() #현재 컴퓨터 크롬드라이버 위치로 변경
#driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)

# Chrome 옵션 설정
options = Options()
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# WebDriver 실행 (ChromeDriver 버전이 Chrome 브라우저 버전과 일치하는지 확인)
#driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)


# 네이버 로그인 화면 이동
login_url = 'https://nid.naver.com/nidlogin.login'
driver.get(login_url)
driver.implicitly_wait(5)

# 아이디& 비밀번호 입력
my_id = 'id' # "id" 대신에 자신의 네이버 아이디 입력
my_pw = 'password' # "password" 대신에 자신의 네이버 비밀번호 입력

# 로그인 id, pw 입력
# 네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 함수를 사용
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# '로그인' 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="log.login"]').click()
time.sleep(random.uniform(1,1.7))
url = 'https://cafe.naver.com/robotclear' # 크롤링할 카페 url 입력
driver.get(url)
time.sleep(1)

In [ ]:
# 날짜 수집 함수
def extract_date(soup):
    try:
        date_element = soup.select_one('span.date')
        if date_element:
            date_text = date_element.text.strip()
            # 날짜 형식 처리
            if "시간 전" in date_text or "분 전" in date_text or "방금" in date_text:
                # 상대적 시간은 현재 날짜로 처리
                return time.strftime('%Y-%m-%d', time.localtime())
            elif "." in date_text:
                # YYYY.MM.DD 형식 -> YYYY-MM-DD로 변환
                return date_text.replace(".", "-")
        return "날짜 없음"  # 기본값
    except Exception as e:
        logging.error(f"날짜 추출 오류: {e}")
        return "날짜 없음"

# 본문 수집 함수
def extract_content(soup, include_images=True):
    contents = []

    # 다양한 본문 선택자 처리
    selectors = [
        'div.se-module.se-module-text',  # 일반 텍스트 본문
        'div.ContentRenderer',           # 특수 서식 본문
        'div.scrap_added',               # 스크랩 본문
    ]
    for selector in selectors:
        elements = soup.select(selector)
        if elements:
            contents.extend([element.text.strip() for element in elements])

    # 이미지 포함 여부에 따른 처리
    if include_images:
        images = soup.select('img')
        for img in images:
            if img.get('src'):
                contents.append(f"[이미지: {img['src']}]")  # 이미지 URL 포함

    return ' '.join(contents).strip() if contents else "본문 없음"

In [ ]:
import pandas as pd

# 제목, 본문, 댓글, 날짜, 검색 키워드, 게시판 이름 - 빈 list 생성
titles = []
reviews = []
comments = []
dates = []
search_keywords = []
board_titles = []
urls = []  # 게시글 URL 저장

time.sleep(random.uniform(1, 1.7))
url = 'https://cafe.naver.com/robotclear'  # 크롤링할 카페 url 입력
driver.get(url)
time.sleep(1)

time.sleep(1)
for k in tqdm(keyword):
    print(k, "키워드 크롤링 중입니다")

    # 검색
    search_box = driver.find_element(By.XPATH, '//*[@id="topLayerQueryInput"]')
    search_box.send_keys(k)
    search_box.send_keys(Keys.RETURN)
    time.sleep(1)

    driver.switch_to.frame('cafe_main')

    # 데이터 수집기간 설정
    time_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="currentSearchDateTop"]')))
    driver.execute_script("arguments[0].click();", time_box)

    # 시작 날짜 입력
    start_time = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="input_1_top"]')))
    start_time.click()
    time.sleep(1)
    start_time.send_keys('20200101')

    # 종료 날짜 입력
    end_time = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="input_2_top"]')))
    end_time.click()
    time.sleep(1.5)
    end_time.send_keys('20230912')

    # 설정 버튼 클릭
    btn_set_top = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_set_top"]')))
    driver.execute_script("arguments[0].click();", btn_set_top)
    time.sleep(1.5)

    # 검색 버튼 클릭
    search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main-area"]/div[1]/div[1]/form/div[4]/button')))
    driver.execute_script("arguments[0].click();", search_button)
    time.sleep(1.5)

    try:
        if driver.find_element(By.XPATH, '//*[@id="main-area"]/div[5]/table/tbody/tr/td/div').text.strip() == '등록된 게시글이 없습니다.':
            driver.get(url)
            time.sleep(1)
            continue
    except:
        print("크롤링 시작합미당~")
        pass

    # 50개씩 보기
    driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]').click()
    driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

    # url 가져오기
    page_url_list = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a.article')]
    first_page = driver.find_element(By.CSS_SELECTOR, 'a.on').get_attribute('href')

    # 페이지 넘기기 검색시 최대 100페이지
    for j in range(1, 41):
        try:
            if j % 8 == 0:
                # 다음 링크 가져오기
                time.sleep(1)
                link = first_page[:-1] + str(j)

                driver.quit()
                driver = webdriver.Chrome()  # 현재 컴퓨터 크롬드라이버 위치로 변경

                # 재로그인
                login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
                driver.get(login_url)
                driver.implicitly_wait(10)

                driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
                driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
                time.sleep(1)

                # '로그인' 버튼 클릭
                driver.find_element('id', 'log.login').click()
                time.sleep(1)

                # 다음 링크부터 가져오기
                driver.get(link)
                driver.switch_to.frame('cafe_main')

        except:
            pass

        try:
            if driver.find_element(By.XPATH, '//*[@id="main-area"]/div[5]/table/tbody/tr/td/div').text.strip() == '등록된 게시글이 없습니다.':
                break
        except:
            pass

        if j > 1:
            next_page = first_page[:-1] + str(j)
            driver.get(next_page)
            time.sleep(1)
            driver.switch_to.frame('cafe_main')

        page_url_list = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a.article')]

        for url in page_url_list:
            try:
                driver.get(url)
                time.sleep(random.uniform(1, 1.7))
                driver.switch_to.frame('cafe_main')
            except:
                continue

            try:
                # 게시판 이름 수집
                board_title = driver.find_element(By.CSS_SELECTOR, 'div.ArticleTitle').text
                board_titles.append(board_title)

                # 제목 수집
                title = driver.find_element(By.CSS_SELECTOR, 'h3.title_text').text
                titles.append(title)

                # 날짜 수집 (함수 적용)
                soup = bs(driver.page_source, 'lxml')
                day = extract_date(soup)
                dates.append(day)

                # 본문 수집 (함수 적용)
                content = extract_content(soup)
                reviews.append(content)

                # 키워드 수집
                search_keywords.append(k)

                # URL 수집
                urls.append(url)

            except:
                continue

            # 댓글 수집
            try:
                iscomment = soup.find_all('span', class_='text_comment')
                if len(iscomment) == 0:
                    comment = '댓글 없음'
                else:
                    comment = '" '.join([c.text.strip() for c in iscomment])  # 댓글을 큰따옴표로 구분
                comments.append(comment)

            except Exception as e:
                logging.error(f"댓글 수집 중 오류 발생: {e}")
                continue

    driver.get(url)
    time.sleep(1)

# DataFrame 생성
data = {
    'date': dates,
    'keyword': search_keywords,
    'title': titles,
    'contents': reviews,
    'comments': comments,
    'board_titles': board_titles,
    'url': urls,
}

df = pd.DataFrame(data)

In [ ]:
# DataFrame 생성
data = {
    'date': dates,
    'keyword': search_keywords,
    'title': titles,
    'contents': reviews,
    'comments': comments,
    'board_titles': board_titles,
    'url': urls,
}

df = pd.DataFrame(data)

In [ ]:
df

### url 기준으로 중복 제거 및 저장

In [ ]:
#URL 기준 필터링
df = df.drop_duplicates(subset='url',keep='first').reset_index(drop=True)

In [ ]:
df

### csv 저장 시 이름 꼭 변경해주세요!!

In [ ]:
df.to_csv("카페명_카테고리명_키워드.csv", index=False, encoding = 'utf-8-sig')